## Example notebook submission
This notebook is a successful submission. It was submitted and scored without any issue.

In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd
import polars as pl
import lightgbm as lgb

import kaggle_evaluation.jane_street_inference_server

In [ ]:
# model uploaded via kaggle UI
path_to_model = "/kaggle/input/test_lgbm_null_to_0/other/default/1/jane_lgbm.txt"

In [ ]:
train_feature_list = ["time_id", "symbol_id"] + [f"feature_{idx:02d}" for idx in range(79)]

In [ ]:
# load saved model to make predictions
model = lgb.Booster(model_file=path_to_model)

In [ ]:
def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    # All the responders from the previous day are passed in at time_id == 0. We save them in a global variable for access at every time_id.
    # Use them as extra features, if you like.
    global lags_
    if lags is not None:
        lags_ = lags
    
    # Replace this section with your own predictions
    y_pred = model.predict(test.select([col for col in train_feature_list]))

    predictions = test.select(pl.col("row_id"))
    predictions = predictions.with_columns(pl.Series("responder_6", y_pred))

    if isinstance(predictions, pl.DataFrame):
        assert predictions.columns == ['row_id', 'responder_6']
    elif isinstance(predictions, pd.DataFrame):
        assert (predictions.columns == ['row_id', 'responder_6']).all()
    else:
        raise TypeError('The predict function must return a DataFrame')
    # Confirm has as many rows as the test data.
    assert len(predictions) == len(test)

    return predictions

In [ ]:
inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/jane-street-realtime-marketdata-forecasting/test.parquet',
            '/kaggle/input/jane-street-realtime-marketdata-forecasting/lags.parquet',
        )
    )